<a href="https://colab.research.google.com/github/asmita-mukherjee/Closing-price-prediction-of-Indian-Stock-/blob/main/Causal_lm_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get the Data from kaggle

In [1]:
!pip install -q opendatasets
!pip install -q torch==2.1.2

In [2]:
import opendatasets as od

In [3]:
# download custom tokenizer and vocab
od.download("https://www.kaggle.com/datasets/asmitamukh/tinystories-custom-bpe")

#download the train and validation dataset
od.download("https://www.kaggle.com/datasets/kaushaltiwari/tiny-shakespeare")

Skipping, found downloaded files in "./tinystories-custom-bpe" (use force=True to force download)
Skipping, found downloaded files in "./tiny-shakespeare" (use force=True to force download)


In [4]:
import random
random.seed(2023)

In [5]:
import logging
logging.basicConfig(format="%(asctime)s - %(message)s",level=logging.INFO,force=True)

In [6]:
!pip install -q torchinfo wandb

In [7]:
from torchinfo import summary

In [8]:
import torch

In [9]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [10]:
import torch
import torch.nn as nn

In [11]:
import numpy as np

In [12]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import wandb
wandb.login(key="b9fca33ebeeea7e66c03c7145a1072cc1bb07412")

2024-01-14 09:58:10,729 - NumExpr defaulting to 2 threads.
wandb: Currently logged in as: asmitaxyz. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
BATCH_SIZE=2

In [15]:
torch.set_default_dtype(torch.float16)

## helper functions

In [16]:
# Read the vocab and merge rules

with open("/content/tinystories-custom-bpe/merge_rules_tiny_shakespeare.pkl","rb") as f:
    merge_rules = pickle.load(f)

with open("/content/tinystories-custom-bpe/vocab_tiny_shakespeare.pkl","rb") as f:
    vocab = pickle.load(f)

In [17]:
vocab

{'gentlewoman',
 'inte',
 'flattering',
 'moiety',
 'seat',
 'prick',
 'marve',
 'd',
 'saints',
 'below',
 'richmond',
 'trouble',
 'cousins',
 'ar',
 'seeing',
 'perforce',
 'disdain',
 "'twere",
 'ad',
 'took',
 'incensed',
 'six',
 'sacra',
 '--',
 'bohemia',
 'hers',
 'adie',
 'heavi',
 'asha',
 'flock',
 'folk',
 'substit',
 'come',
 'lancaster',
 'hun',
 'graves',
 'al',
 'cun',
 'enjoy',
 'sho',
 'ze',
 'flourish',
 'orrow',
 'physic',
 'entertain',
 'joints',
 'charges',
 'sky',
 'soldi',
 'strong',
 'affair',
 'flood',
 'secure',
 'maje',
 'offices',
 'bold',
 'cous',
 'myself',
 'grain',
 'cen',
 'sick',
 'blind',
 'burgundy',
 'along',
 'nup',
 'grim',
 'puts',
 'forthwit',
 'marg',
 'destruc',
 'sme',
 'mightst',
 'adri',
 'letters',
 'got',
 'fury',
 'fal',
 'outward',
 'thinks',
 'dish',
 'richmon',
 'fur',
 'mercy',
 'ond',
 'lt',
 'plague',
 'dur',
 'stood',
 'behavi',
 'bad',
 'cab',
 'bestow',
 'conquer',
 'iiii',
 'kit',
 'govern',
 'wave',
 'nat',
 'va',
 'step',
 

In [18]:
torch.__version__

'2.1.2+cu121'

In [19]:
vocab.add("</s>")
vocab.add("<unk>")

In [20]:
def tokenize(text,merge_rules=merge_rules):
    '''Return tokens from the text'''
    text = text.lower() # our vocabulary is uncased.
    words = nltk.word_tokenize(text)

    ##preparing dict where word is the key and the tokens is its value
    word_dict = dict()
    for word in words:
        word_dict[word] = []
        for char in word:
            word_dict[word].append(char)

    for merge_rule in merge_rules.keys():
        #go through each merge rule and tokenize each word from left to right
        for word in word_dict.keys():
            tokens = word_dict[word]
            idx=0
            while(idx<len(tokens)-1):
                pair = (tokens[idx],tokens[idx+1])
                if pair==merge_rule:
                    #merge the token of the word as per the merge rule
                    #if merged we dont increase the index of the tokens because the next pair should include the new pair and the next char
                    tokens = tokens[:idx]+[tokens[idx]+tokens[idx+1]]+tokens[idx+2:]
                    word_dict[word] = tokens
                else:
                    #if not merged then we slide the window over
                    idx = idx+1

    tokens = [token for item in word_dict.values() for token in item ]
    return tokens

In [21]:

def encode(list_of_tokens):
  vocab_l = list(vocab)
  token_tensor = []
  for token in list_of_tokens:
    try:
      token_tensor.append(vocab_l.index(token))
    except ValueError:
      logging.info(f"token {token} not in vocab")

  return token_tensor


def decode(list_of_idx):
  tokens = []
  vocab_l = list(vocab)
  for idx in list_of_idx:
    tokens.append(vocab_l[idx])
  return tokens

## Load dataset

In [22]:
text = ""
with open("/content/tiny-shakespeare/tiny-shakespeare.txt","r") as f:
  text = f.read()

In [23]:
tokenized_data = tokenize(text)
tokenized_data[:5]

['first', 'citizen', ':', 'before', 'we']

In [24]:
encoded_text = encode(tokenized_data)
encoded_text[:5]

2024-01-14 09:59:06,068 - token & not in vocab
2024-01-14 09:59:06,935 - token $ not in vocab


[1094, 980, 3778, 1045, 3832]

In [25]:
decode(encoded_text[:5])

['first', 'citizen', ':', 'before', 'we']

In [26]:
context_length = 8

#chunking windows of 8 from the text
chunks = []


list_of_inputs = []
list_of_labels = []

for idx in range(0,len(encoded_text),context_length+1):
  list_of_inputs.append(encoded_text[idx:idx+context_length])

  if idx+1>len(encoded_text):
    list_of_labels.append(vocab.index("</s>"))
  else:
    list_of_labels.append(encoded_text[idx+1:idx+context_length+1])

    if idx+context_length>len(encoded_text):
      list_of_labels[-1].extend(vocab.index("</s>"))

vocab.add("</s>")

In [27]:
list_of_inputs[:5]

[[1094, 980, 3778, 1045, 3832, 3603, 2037, 3404],
 [1768, 4685, 2693, 1612, 2895, 1034, 1132, 1617],
 [3791, 1344, 4135, 362, 2240, 1589, 358, 4111],
 [262, 2532, 2016, 1025, 1305, 1681, 3958, 3239],
 [959, 2826, 2023, 3918, 4079, 3920, 2379, 1140]]

In [28]:
list_of_labels[:5]

[[980, 3778, 1045, 3832, 3603, 2037, 3404, 2207],
 [4685, 2693, 1612, 2895, 1034, 1132, 1617, 1328],
 [1344, 4135, 362, 2240, 1589, 358, 4111, 4357],
 [2532, 2016, 1025, 1305, 1681, 3958, 3239, 3642],
 [2826, 2023, 3918, 4079, 3920, 2379, 1140, 385]]

In [29]:
#shuffle the list of inputs and labels and split into train and test set
temp = list(zip(list_of_inputs,list_of_labels))
random.shuffle(temp)
list_of_inputs,list_of_labels = zip(*temp)
list_of_inputs,list_of_labels = list(list_of_inputs),list(list_of_labels)

In [30]:
len_of_train = int((0.9)*len(list_of_inputs))
train_inputs,train_labels = list_of_inputs[:len_of_train],list_of_labels[:len_of_train]
test_inputs,test_labels = list_of_inputs[len_of_train:],list_of_labels[len_of_train:]

In [31]:
len(train_inputs),len(train_labels)

(2914, 2914)

In [32]:
len(test_inputs),len(test_labels)

(324, 324)

> Divide into Batches

In [33]:
batch_inputs_train = []
batch_labels_train = []

for idx in range(0,len(train_inputs),BATCH_SIZE):
  batch_inputs_train.append(train_inputs[idx:idx+BATCH_SIZE])
  batch_labels_train.append(train_labels[idx:idx+BATCH_SIZE])

batch_inputs_train = torch.tensor(batch_inputs_train,dtype=torch.long)
batch_labels_train = torch.tensor(batch_labels_train,dtype=torch.long)

In [34]:
batch_inputs_train[0].shape

torch.Size([2, 8])

In [35]:
batch_inputs_train.shape,batch_labels_train.shape

(torch.Size([1457, 2, 8]), torch.Size([1457, 2, 8]))

In [36]:
batch_inputs_test = []
batch_labels_test = []

for idx in range(0,len(test_inputs),BATCH_SIZE):
  batch_inputs_test.append(test_inputs[idx:idx+BATCH_SIZE])
  batch_labels_test.append(test_labels[idx:idx+BATCH_SIZE])

batch_inputs_test = torch.tensor(batch_inputs_test,dtype=torch.long)
batch_labels_test = torch.tensor(batch_labels_test,dtype=torch.long)

batch_inputs_test.shape,batch_labels_test.shape

(torch.Size([162, 2, 8]), torch.Size([162, 2, 8]))

In [37]:
batch_inputs_train= batch_inputs_train.to(device)
batch_labels_train= batch_labels_train.to(device)
batch_inputs_test= batch_inputs_test.to(device)
batch_labels_test= batch_labels_test.to(device)

In [38]:
class BareLLM(nn.Module):
    def __init__(self,vocab_size,embed_dim,hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        #self.layer_norm = nn.LayerNorm(embed_dim)
        self.Linear1 = nn.Linear(embed_dim,hidden_dim)
        self.relu1 = nn.ReLU()
        self.Linear2 = nn.Linear(hidden_dim,vocab_size)
        self.relu2 = nn.ReLU()

    def forward(self,input_tensor):
        logging.debug(f"Shape of input tensor {input_tensor.shape}")
        embedded_tensor = self.embedding(input_tensor)
        logging.debug(f"Shape of embedding matrix {embedded_tensor.shape}")
        #norm_tensor = self.layer_norm(embedded_tensor)
        #logging.debug(f"Shape of matrix after norm {norm_tensor.shape}")
        linear_output_1 = self.Linear1(embedded_tensor)
        logging.debug(f"Shape of tensor after linear layer 1 {linear_output_1.shape}")
        relu_output_1 = self.relu1(linear_output_1)
        linear_output_2 = self.Linear2(relu_output_1)
        logging.debug(f"Shape of tensor after linear layer 2 {linear_output_2.shape}")
        relu_output_2 = self.relu2(linear_output_2)
        return relu_output_2

    def generate(self,starting_idx,max_new_tokens):
      '''
        Autoregressively generates a string upto max_new_tokens starting from starting idx
      '''
      gen_tokens = 0

      idx = starting_idx

      while gen_tokens<max_new_tokens:
        next_token_logits = self(idx)
        logging.debug(f"Shape of next token logits {next_token_logits.shape}")
        #only take the logit of the last token of the input
        next_token_logits = next_token_logits[:,-1,:]
        logging.debug(f"Shape of next token logits of the last token {next_token_logits.shape}")
        prob_of_next_token = torch.nn.functional.softmax(next_token_logits,dim=1)
        logging.debug(f"Prob of next token {prob_of_next_token.shape}")#1*vocab_size
        next_token_idx = torch.topk(prob_of_next_token,dim=1,k=3)
        next_token_idx = torch.multinomial(next_token_idx.values,1)
        logging.debug(f"Next token idx {next_token_idx}")
        idx = torch.cat((idx,next_token_idx),dim=1)
        gen_tokens = gen_tokens+1

      return idx






In [39]:
model = BareLLM(vocab_size=len(vocab),embed_dim=768,hidden_dim=128)
model = model.to(device)

In [40]:
gen_tokens = model.generate(torch.tensor([[1458]],dtype=torch.long,device=device),200)

In [41]:
text_bef_training = ' '.join(decode(list(gen_tokens[0])))

In [42]:
summary(model)

Layer (type:depth-idx)                   Param #
BareLLM                                  --
├─Embedding: 1-1                         3,607,296
├─Linear: 1-2                            98,432
├─ReLU: 1-3                              --
├─Linear: 1-4                            605,913
├─ReLU: 1-5                              --
Total params: 4,311,641
Trainable params: 4,311,641
Non-trainable params: 0

In [43]:
torch.cuda.empty_cache()

In [44]:
print(model.parameters())

<generator object Module.parameters at 0x7bd79aae2d50>


In [45]:
model = model.to(device)

> Checking forward pass for one iteration

In [46]:

s_output = model(torch.tensor([0],dtype=torch.long,device=device))

In [47]:
s_output = model(batch_inputs_train[0])

In [48]:
print(s_output.shape)

torch.Size([2, 8, 4697])


### Write Training Loop

In [49]:
torch.cuda.empty_cache()

In [50]:
vocab_size = len(vocab)
vocab_size

4697

In [51]:
optimizer = torch.optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

def train_one_epoch(input_batch,label_batch):
    running_loss = 0 #captures the loss at each iteration of the dataset
    last_loss = 0 #captures the loss at the training of the epoch

    for batch_nos,(inputs,labels) in enumerate(zip(input_batch,label_batch)):
        logging.debug(f"Batch nos {batch_nos}")
        #at the start of every batch zero grad optimizer to clean the gradients
        optimizer.zero_grad()

        #output for the batch
        logits = model(inputs)

        #compute loss
        #reshaping the logits to batch_size*vocab_size*context_length for pytorch nn.cross_entropy
        batch_size,context_len,vocab_size = logits.shape
        logits = logits.reshape(batch_size,vocab_size,context_len)
        loss = loss_fn(logits,labels)
        loss.backward()

        #apply the updates to all parameters
        optimizer.step()

        running_loss += loss.item()
        if batch_nos%100 ==99:
            last_loss = running_loss/500
            logging.info(f"For batch {batch_nos+1} loss at 500 samples is {last_loss}")
            running_loss = 0

    return last_loss


In [52]:
wandb.init(project="bare_min_llm_14th_jan",name="correct_causal_lm_14th_jan")
wandb.watch(model, log_freq=10)

[]

In [53]:
batch_inputs_train.device

device(type='cuda', index=0)

In [54]:
EPOCH = 10000

for epoch in range(EPOCH):
    logging.info(f"Starting training at {epoch+1}")
    model.train(True) #making sure gradient tracking is on
    avg_loss = train_one_epoch(batch_inputs_train,batch_labels_train)

    running_val_loss = 0.0

    model.eval() #set the model to eval model(handels dropout calc etc)
    with torch.no_grad():
        for i,(v_input,v_label) in enumerate(zip(batch_inputs_test,batch_labels_test)):
            v_input = torch.tensor(v_input,dtype=torch.long,device=device)
            v_label = torch.tensor(v_label,dtype=torch.long,device=device)
            v_output = model(v_input)
            B,T,C = v_output.shape
            v_output = v_output.reshape(B,C,T)
            v_loss = loss_fn(v_output,v_label)
            running_val_loss += v_loss

    avg_vloss = running_val_loss / (epoch + 1)
    wandb.log({"train_loss":avg_loss})
    wandb.log({"val_loss":avg_vloss})
    wandb.log({"epoch":epoch})
    logging.info('LOSS train {} valid {}'.format(avg_loss, avg_vloss))





2024-01-14 09:59:12,611 - Starting training at 1
2024-01-14 09:59:13,664 - For batch 100 loss at 500 samples is 1.690265625
2024-01-14 09:59:14,544 - For batch 200 loss at 500 samples is 1.680625
2024-01-14 09:59:15,515 - For batch 300 loss at 500 samples is 1.663171875
2024-01-14 09:59:16,404 - For batch 400 loss at 500 samples is 1.6590234375
2024-01-14 09:59:17,297 - For batch 500 loss at 500 samples is 1.6510234375
2024-01-14 09:59:18,423 - For batch 600 loss at 500 samples is 1.6391875
2024-01-14 09:59:19,794 - For batch 700 loss at 500 samples is 1.628140625
2024-01-14 09:59:21,069 - For batch 800 loss at 500 samples is 1.622671875
2024-01-14 09:59:21,957 - For batch 900 loss at 500 samples is 1.619625
2024-01-14 09:59:22,812 - For batch 1000 loss at 500 samples is 1.61190625
2024-01-14 09:59:23,688 - For batch 1100 loss at 500 samples is 1.606671875
2024-01-14 09:59:24,575 - For batch 1200 loss at 500 samples is 1.5984765625
2024-01-14 09:59:25,448 - For batch 1300 loss at 500 s

KeyboardInterrupt: 

## Inferencing

In [55]:
'.'in vocab

True

In [56]:
list(vocab).index('.')

1612

In [57]:
generated_tokens = model.generate(torch.tensor([[1458]],dtype=torch.long,device=device),max_new_tokens=200)

In [58]:
text_after_training = ' '.join(decode(list(generated_tokens[0])))

In [59]:
print(f"Text Before Training -->\n {text_bef_training}")

Text Before Training -->
 gonz inte gentlewoman inte inte inte flattering inte flattering inte gentlewoman flattering flattering inte inte gentlewoman gentlewoman inte gentlewoman gentlewoman inte gentlewoman flattering inte inte gentlewoman gentlewoman gentlewoman inte gentlewoman flattering flattering gentlewoman gentlewoman inte gentlewoman gentlewoman flattering flattering inte flattering inte inte inte inte gentlewoman inte flattering gentlewoman gentlewoman gentlewoman inte flattering flattering inte inte gentlewoman inte flattering flattering flattering gentlewoman inte inte flattering inte flattering gentlewoman inte flattering gentlewoman inte inte gentlewoman gentlewoman inte flattering flattering flattering gentlewoman flattering inte gentlewoman gentlewoman flattering flattering inte flattering gentlewoman flattering inte inte gentlewoman flattering gentlewoman flattering inte inte flattering inte inte gentlewoman gentlewoman gentlewoman inte inte gentlewoman gentlewoman in

In [60]:
print(f"Text After Traning --> \n {text_after_training}")

Text After Traning --> 
 gonz gentlewoman gentlewoman flattering gentlewoman flattering inte gentlewoman gentlewoman gentlewoman gentlewoman flattering flattering gentlewoman gentlewoman inte inte inte flattering gentlewoman flattering gentlewoman flattering inte inte inte flattering flattering flattering gentlewoman inte gentlewoman gentlewoman flattering flattering flattering flattering flattering inte flattering inte gentlewoman flattering inte inte inte flattering gentlewoman flattering flattering gentlewoman flattering inte flattering flattering inte gentlewoman inte inte flattering gentlewoman inte gentlewoman inte inte inte flattering gentlewoman flattering gentlewoman flattering flattering flattering flattering gentlewoman gentlewoman inte inte flattering inte flattering gentlewoman gentlewoman gentlewoman inte gentlewoman inte gentlewoman gentlewoman flattering flattering gentlewoman flattering inte gentlewoman flattering gentlewoman gentlewoman inte inte flattering flattering